In [1]:
import pandas as pd
import numpy as np
from scipy.stats import linregress
from sklearn.preprocessing import MinMaxScaler

# =======================================================
# 1. 파일 불러오기
# =======================================================
print("📂 데이터 불러오는 중...")

# (1) 원본 VIP 고객 데이터 (상림이가 방금 올린 파일!)
df = pd.read_csv("vip_customers.csv")

# (2) 우리가 선정한 150개 변수 리스트
features_df = pd.read_csv("Final_Selected_150_Features_SHAP_Priority.csv")
selected_features = features_df['변수명'].tolist()

print(f"✅ 원본 데이터 크기: {df.shape}")
print(f"✅ 선정된 변수 개수: {len(selected_features)}개")


# =======================================================
# 2. Churn Score (이탈 위험 점수) 생성 로직
# =======================================================
print("\n⚡ 이탈 위험 점수(Churn_Score) 계산 시작...")

# 데이터 정렬 (고객별, 날짜순) - 추세를 구하려면 순서가 중요해!
df = df.sort_values(by=['발급회원번호', '기준년월'])

# (함수) 6개월 기울기 계산
def calc_slope_6m(series):
    if len(series) < 2 or np.sum(series) == 0: return 0
    y = series.values
    x = np.arange(len(y))
    if np.all(y == y[0]): return 0 # 변화 없으면 0
    slope, _, _, _, _ = linregress(x, y)
    if np.isnan(slope): return 0
    return slope

# 1) 주요 지표 3가지에 대해 기울기(Slope) 구하기
print("   - 소비/잔액/빈도 추세 계산 중...")
df['Slope_Spend'] = df.groupby('발급회원번호')['이용금액_신용_B0M'].transform(calc_slope_6m)
df['Slope_Count'] = df.groupby('발급회원번호')['이용건수_신용_B0M'].transform(calc_slope_6m)

# 잔액 컬럼명 찾기 (혹시 이름 다를까봐 안전장치!)
balance_col = '잔액_B0M' if '잔액_B0M' in df.columns else [c for c in df.columns if '잔액' in c][0]
df['Slope_Balance'] = df.groupby('발급회원번호')[balance_col].transform(calc_slope_6m)

# 2) 가장 최근 월(Target Month)만 남기기
target_month = df['기준년월'].max()
df_final = df[df['기준년월'] == target_month].copy()
print(f"   - 분석 기준월: {target_month} (고객 수: {len(df_final)}명)")

# 3) 기울기를 점수(0~1)로 변환 (감소폭이 클수록 위험!)
scaler = MinMaxScaler()
def convert_slope_to_score(df_in, col_name):
    # 음수(감소)인 경우만 양수로 뒤집어서 점수화 (많이 줄어들수록 고위험)
    negative_slopes = df_in[col_name].apply(lambda x: -x if x < 0 else 0)
    return scaler.fit_transform(negative_slopes.values.reshape(-1, 1)).flatten()

score_spend = convert_slope_to_score(df_final, 'Slope_Spend')
score_balance = convert_slope_to_score(df_final, 'Slope_Balance')
score_count = convert_slope_to_score(df_final, 'Slope_Count')

# 4) 가중치 적용해서 최종 점수(100점 만점) 만들기
# 비중: 소비(40) > 잔액(30) > 빈도(20) > 리스크(10)
W_SPEND, W_BALANCE, W_COUNT, W_RISK = 40, 30, 20, 10

# 리스크 점수 (연체나 거절 있으면 1점 -> 10점 부여)
has_risk = np.where(
    (df_final.get('연체잔액_B0M', 0) > 0) | (df_final.get('승인거절건수_B0M', 0) > 0), 
    1, 0
)

df_final['Churn_Score'] = (
    (score_spend * W_SPEND) + 
    (score_balance * W_BALANCE) + 
    (score_count * W_COUNT) + 
    (has_risk * W_RISK)
)
df_final['Churn_Score'] = df_final['Churn_Score'].round(1)

print(f"✅ Churn_Score 생성 완료! (평균: {df_final['Churn_Score'].mean():.1f}점)")


# =======================================================
# 3. 150개 변수만 뽑아서 저장하기
# =======================================================
print("\n✂️ 150개 핵심 변수만 추출 중...")

# 최종 저장할 컬럼: [ID] + [150개 변수] + [Target(점수)]
# (혹시 리스트에 있는데 데이터에 없는 변수가 있을까봐 교집합으로 처리)
final_cols = ['발급회원번호'] + [col for col in selected_features if col in df_final.columns] + ['Churn_Score']

df_export = df_final[final_cols]

# 파일 저장
save_path = "VIP_Target_150_Selected.csv"
df_export.to_csv(save_path, index=False, encoding='utf-8-sig')

print(f"\n🎉 모든 작업 완료!")
print(f"📂 저장된 파일명: {save_path}")
print(f"📊 최종 데이터 크기: {df_export.shape}")

📂 데이터 불러오는 중...


/var/folders/3p/8hc9c44s38j902ytq253kt6m0000gn/T/ipykernel_1898/2345645296.py:12: DtypeWarning: Columns (339) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("vip_customers.csv")


✅ 원본 데이터 크기: (11460, 822)
✅ 선정된 변수 개수: 150개

⚡ 이탈 위험 점수(Churn_Score) 계산 시작...
   - 소비/잔액/빈도 추세 계산 중...
   - 분석 기준월: 201812 (고객 수: 1910명)
✅ Churn_Score 생성 완료! (평균: 9.0점)

✂️ 150개 핵심 변수만 추출 중...

🎉 모든 작업 완료!
📂 저장된 파일명: VIP_Target_150_Selected.csv
📊 최종 데이터 크기: (1910, 148)
